In [ ]:
# 여러개의 머신러닝을 가지고 평가할 수 있는 것들.

In [19]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# 평가할 머신러닝 모델을 선정
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
import numpy as np

In [5]:
iris = load_iris()
x = iris.data
y = iris.target

In [6]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=100)

In [17]:
# 5번 실행해서 평균 성능을 취득하는 것. 교차검증
from sklearn.model_selection import cross_validate

In [38]:
pipeline=Pipeline([
    ('scaler',StandardScaler()),
    ('LogisticRegression',LogisticRegression())
])
scores = cross_validate(pipeline,x_train,y_train,n_jobs=-1)
# pipeline.fit(x_train,y_train)
np.mean(scores['test_score'])

0.9470355731225297

In [13]:
pipeline.score(x_test,y_test)

0.9473684210526315

In [ ]:
#1 pipline을 이용해 전처리부터 -> 모델학습하는 과정까지 일괄처리 가능!
#2 교차검증을 통해 우연히 좋은 성능을 발휘하는 모델을 방지하기 위해 교차검증함.
# 1번과 2번의 과정을 지정한 머신러닝 모델에 대해 각각적용하고 가장 최상의 모델을 선택한다.

In [21]:
# 1. pipline을 이용해서 전처리부터->모델학습하는 과정까지 일괄처리
# 2. 교차검증을 통해 우연히 좋은 성능을 발위하는 모델을 방지
# 1번과 2번의 과정을 지정한 머신러닝 모델에대해 각각 적용하고 가장 최상의 모델을 선택
score_list = []
for model in [LogisticRegression(), DecisionTreeClassifier(), SGDClassifier()]:
  pipeline = Pipeline([
    ('scaler',StandardScaler()),
    ('classifier',model)    
  ])
  scores = cross_validate(pipeline,x_train,y_train,n_jobs=-1)
  m = np.mean(scores["test_score"])
  score_list.append(m)
  print(f'model : {model} mean test score : {m}')


model : LogisticRegression() mean test score : 0.9470355731225297
model : DecisionTreeClassifier() mean test score : 0.9644268774703558
model : SGDClassifier() mean test score : 0.9466403162055336


In [ ]:
# 최상의 모델을 선택한 후... 하이퍼 파라메터 튜닝을 통해 다시 최상의 점수를 획득
# 이렇게 해도 별로 나아지거나 향상된 기미가 안 보일 때는
# 처음 데이터 수집 및 처리과정을 다시 한 번 검토(피처 엔지니어링을 통해)

# 피처 엔지니어링
# p_value, VIF 계수 확인을 통한 적절한 feature만 골라서 다시 학습.
# 도메인지식을 활용
# 트리계열 머신러닝을 먼저 적용해서. feature 선택
# 범주형 데이터 one hot 표현하고, 기존 컬럼을 분석해서 새로운 컬럼을 도출... 나이...(연속된 데이터)-> 범주형 데이터로 새로 생성.

# 기존에 없는 모델이나 라이브러리를 각종 논문이나, 대회 및 기타등등 찾아내서 적용.

AutoML
  사이킷런 계열 : TPOT

In [ ]:
!pip install tpot

In [24]:
from tpot import TPOTClassifier
automl = TPOTClassifier(generations=10)
automl.fit(x_train,y_train)

TPOTClassifier(generations=10)

타 계열의 AutoML
  pycaret

In [ ]:
!pip install pycaret

In [26]:
from pycaret import classification
classification.setup(data=x_train, target = y_train)

,Description,Value
0,Session id,8343
1,Target,target
2,Target type,Multiclass
3,Original data shape,"(112, 5)"
4,Transformed data shape,"(112, 5)"
5,Transformed train set shape,"(78, 5)"
6,Transformed test set shape,"(34, 5)"
7,Numeric features,4
8,Preprocess,True
9,Imputation type,simple


In [29]:
best= classification.compare_models(n_select=5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
qda,Quadratic Discriminant Analysis,0.9875,1.0000,0.9875,0.9917,0.9875,0.9814,0.9833,0.0590
lda,Linear Discriminant Analysis,0.9875,0.9954,0.9875,0.9917,0.9875,0.9814,0.9833,0.0590
knn,K Neighbors Classifier,0.9750,0.9854,0.9750,0.9812,0.9735,0.9614,0.9656,0.0830
lr,Logistic Regression,0.9625,0.9908,0.9625,0.9729,0.9601,0.9424,0.9485,0.1470
ada,Ada Boost Classifier,0.9625,0.9862,0.9625,0.9729,0.9601,0.9424,0.9485,0.1350
et,Extra Trees Classifier,0.9625,0.9896,0.9625,0.9656,0.9613,0.9424,0.9445,0.3800
nb,Naive Bayes,0.9500,0.9862,0.9500,0.9573,0.9488,0.9238,0.9278,0.1120
dt,Decision Tree Classifier,0.9500,0.9633,0.9500,0.9500,0.9500,0.9238,0.9238,0.0960
rf,Random Forest Classifier,0.9500,0.9954,0.9500,0.9562,0.9476,0.9229,0.9271,0.2120
xgboost,Extreme Gradient Boosting,0.9500,0.9908,0.9500,0.9562,0.9476,0.9229,0.9271,0.1830


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [30]:
for model in best:
  print(model)

QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
                              store_covariance=False, tol=0.0001)
LinearDiscriminantAnalysis(covariance_estimator=None, n_components=None,
                           priors=None, shrinkage=None, solver='svd',
                           store_covariance=False, tol=0.0001)
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                     weights='uniform')
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=8343, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
AdaBoostClassifier(algorithm='SAMME.R', base_estimator='deprecated',
                   estimator=None, learning_rate=1.0, n_estimators=50,
                  

In [32]:
best[0].predict_proba(x_test)[0], best[1].predict_proba(x_test)[0]

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but QuadraticDiscriminantAnalysis was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearDiscriminantAnalysis was fitted with feature names
  warnings.warn(


(array([3.85766014e-151, 5.00127488e-006, 9.99994999e-001]),
 array([3.21735618e-48, 7.02942005e-06, 9.99992971e-01]))

In [36]:
models_proba =  np.array([model.predict_proba(x_test) for model in best])
models_mean = np.mean(models_proba,axis=0)
print(models_proba.shape, models_mean.shape)
predict = np.argmax(models_mean,axis=1)s
predict

(5, 38, 3) (38, 3)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but QuadraticDiscriminantAnalysis was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearDiscriminantAnalysis was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


array([2, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 1, 1, 1, 2, 2, 2, 0,
       2, 0, 1, 2, 1, 0, 1, 2, 1, 1, 1, 0, 0, 1, 0, 1])